[Quickdraw](https://github.com/googlecreativelab/quickdraw-dataset)

In [ ]:
# native python libraries imports 
import math
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn imports 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# keras imports 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import get_file

# required for compatibility between sklearn and keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# install keras-tuner
!pip install keras-tuner
from kerastuner.tuners import RandomSearch, BayesianOptimization, Sklearn
from kerastuner.engine.hyperparameters import HyperParameters

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 6.4 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
def load_quickdraw10():
    """
    Fill out this doc string, and comment the code, for practice in writing the kind of code that will get you hired. 
    """
    
    URL_ = "https://github.com/LambdaSchool/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/quickdraw10.npz?raw=true"
    
    path_to_zip = get_file('./quickdraw10.npz', origin=URL_, extract=False)

    data = np.load(path_to_zip)
    
    # normalize 
    max_pixel_value = 255
    X = data['arr_0']/max_pixel_value
    Y = data['arr_1']
        
    return train_test_split(X, Y, shuffle=True)

In [ ]:
X_train, X_test, y_train, y_test = load_quickdraw10()

25436160/25421363 [==============================] - 0s 0us/step


In [ ]:
X_train.shape

(75000, 784)

In [ ]:
y_train.shape

(75000,)



## Tune Hyperperameters w/ GridsearchCV 


In [ ]:
def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
        """
        Generates and returns the number of nodes in each hidden layer. 
        To be clear, this excludes the input and output layer. 

        Note
        ----
        Number of nodes in each layer is linearly incremented. 
        For example, gen_layer_nodes(5, 500, 100) will generate [500, 400, 300, 200, 100]

        Parameters
        ----------
        n_layers: int
            Number of hidden layers
            This values should be 2 or greater 

        first_layer_nodes: int

        last_layer_nodes: int

        Returns
        -------
        layers: list of ints
            Contains number of nodes for each layer 
        """

        # throws an error if n_layers is less than 2 
        assert n_layers >= 2, "n_layers must be 2 or greater"

        layers = []

        if negative_node_incrementation:
            # subtract this amount from previous layer's nodes in order to increment towards smaller numbers 
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
            
        # when set to False number of nodes are increased for subsequent layers
        else:
            # add this amount from previous layer's nodes in order to increment towards larger numbers 
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)

        nodes = first_layer_nodes

        for i in range(1, n_layers+1):

            layers.append(math.ceil(nodes))

            # increment nodes for next layer 
            nodes = nodes + nodes_increment

        return layers


# Function to create model, required for KerasClassifier
def create_model(n_layers,  first_layer_nodes, last_layer_nodes, act_funct ="relu", negative_node_incrementation=True):
    """"
    Returns a compiled keras model 
    
    Parameters
    ----------
    n_layers: int 
        number of hidden layers in model 
        To be clear, this excludes the input and output layer.
        
    first_layer_nodes: int
        Number of nodes in the first hidden layer 

    last_layer_nodes: int
        Number of nodes in the last hidden layer (this is the layer just prior to the output layer)
        
     act_funct: string 
         Name of activation function to use in hidden layers (this excludes the output layer)
        
    Returns
    -------
    model: keras object 
    """

    # create model
    model = Sequential()
    
    n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
    
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=act_funct))
        else:
            model.add(Dense(n_nodes[i-1], activation=act_funct))
            
            
    # output layer 
    model.add(Dense(10, # 10 unit/neurons in output layer because we have 10 possible labels to predict  
                    activation='softmax')) # use softmax for a label set greater than 2            
    
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', # adam is a good default optimizer 
                  metrics=['accuracy'])
    
    # do not include model.fit() inside the create_model function
    # KerasClassifier is expecting a complied model 
    return model


    



In [ ]:
    def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
        """
        Generates and returns the number of nodes in each hidden layer. 
        To be clear, this excludes the input and output layer. 

        Note
        ----
        Number of nodes in each layer is linearly incremented. 
        For example, gen_layer_nodes(5, 500, 100) will generate [500, 400, 300, 200, 100]

        Parameters
        ----------
        n_layers: int
            Number of hidden layers
            This values should be 2 or greater 

        first_layer_nodes: int

        last_layer_nodes: int

        Returns
        -------
        layers: list of ints
            Contains number of nodes for each layer 
        """

        # throws an error if n_layers is less than 2 
        assert n_layers >= 2, "n_layers must be 2 or greater"

        layers = []

        if negative_node_incrementation:
            # subtract this amount from previous layer's nodes in order to increment towards smaller numbers 
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
            #print(f'nodes increment = {nodes_increment}')
            
        # when set to False number of nodes are increased for subsequent layers
        else:
            # add this amount from previous layer's nodes in order to increment towards larger numbers 
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
            #print(f'nodes increment = {nodes_increment}')

        nodes = first_layer_nodes

        for i in range(1, n_layers+1):

            layers.append(math.ceil(nodes))

            # increment nodes for next layer 
            nodes = nodes + nodes_increment

        return layers




### `negative_node_incrementation = True` 

In [ ]:
n_layers = 5
first_layer_nodes = 500
last_layer_nodes = 100
negative_node_incrementation = True
n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
print(f'Number of nodes in successive layers: {n_nodes}')

Number of nodes in successive layers: [500, 400, 300, 200, 100]


### `negative_node_incrementation = False`

In [ ]:
n_layers = 5
first_layer_nodes = 100
last_layer_nodes = 500
negative_node_incrementation = False
n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
print(f'Number of nodes in successive layers: {n_nodes}')

Number of nodes in successive layers: [100, 200, 300, 400, 500]


In [ ]:
model = create_model(n_layers, first_layer_nodes, last_layer_nodes)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 dense_2 (Dense)             (None, 300)               60300     
                                                                 
 dense_3 (Dense)             (None, 400)               120400    
                                                                 
 dense_4 (Dense)             (None, 10)                4010      
                                                                 
Total params: 283,410
Trainable params: 283,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = create_model(n_layers=10, first_layer_nodes=100, last_layer_nodes=500, act_funct='relu', negative_node_incrementation=False)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 100)               78500     
                                                                 
 dense_6 (Dense)             (None, 145)               14645     
                                                                 
 dense_7 (Dense)             (None, 189)               27594     
                                                                 
 dense_8 (Dense)             (None, 234)               44460     
                                                                 
 dense_9 (Dense)             (None, 278)               65330     
                                                                 
 dense_10 (Dense)            (None, 323)               90117     
                                                                 
 dense_11 (Dense)            (None, 367)              

### Hyperparameter search

In [ ]:
param_grid = {'n_layers': [2, 3],
              'epochs': [3], 
              "first_layer_nodes": [500, 300],
              "last_layer_nodes": [100, 50]
             }

In [ ]:
model = KerasClassifier(create_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
%%time
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)

grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Epoch 1/3
1563/1563 [==============================] - 13s 5ms/step - loss: 0.6595 - accuracy: 0.8026
Epoch 2/3
1563/1563 [==============================] - 4s 2ms/step - loss: 0.4459 - accuracy: 0.8668
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.4744 - accuracy: 0.8640
Epoch 1/3
1563/1563 [==============================] - 4s 2ms/step - loss: 0.6627 - accuracy: 0.8027
Epoch 2/3
1563/1563 [==============================] - 4s 3ms/step - loss: 0.4512 - accuracy: 0.8671
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.4684 - accuracy: 0.8653
Epoch 1/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.6621 - accuracy: 0.8017
Epoch 2/3
1563/1563 [==============================] - 4s 2ms/step - loss: 0.4486 - accuracy: 0.8672
Epoch 3/3
782/782 [==============================] - 2s 3ms/step - loss: 0.4663 - accuracy: 0.8617
Epoch 1/3
1563/1563 [===============

In [ ]:
best_model = grid_result.best_estimator_

In [ ]:
best_model.get_params()

{'build_fn': <function __main__.create_model>,
 'epochs': 3,
 'first_layer_nodes': 500,
 'last_layer_nodes': 50,
 'n_layers': 3}

In [ ]:
# use create_model to create a model 
model = create_model(n_layers=2, first_layer_nodes=500, last_layer_nodes=100, act_funct='relu', negative_node_incrementation=True)

In [ ]:
model.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_78 (Dense)            (None, 500)               392500    
                                                                 
 dense_79 (Dense)            (None, 10)                5010      
                                                                 
Total params: 397,510
Trainable params: 397,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# define the grid search parameters
param_grid = {'n_layers': [2, 3],
              'epochs': [3], 
              "first_layer_nodes": [500, 300],
              "last_layer_nodes": [100, 50]
             }

In [ ]:
model = KerasClassifier(create_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
%%time
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)

grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Epoch 1/3
1563/1563 [==============================] - 4s 3ms/step - loss: 0.6598 - accuracy: 0.8037
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4499 - accuracy: 0.8658
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.4629 - accuracy: 0.8674
Epoch 1/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.6687 - accuracy: 0.7997
Epoch 2/3
1563/1563 [==============================] - 4s 3ms/step - loss: 0.4542 - accuracy: 0.8645
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.4669 - accuracy: 0.8633
Epoch 1/3
1563/1563 [==============================] - 4s 3ms/step - loss: 0.6627 - accuracy: 0.8012
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4491 - accuracy: 0.8669
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.4564 - accuracy: 0.8669
Epoch 1/3
1563/1563 [================

In [ ]:
best_model = grid_result.best_estimator_

In [ ]:
best_model.get_params()

{'build_fn': <function __main__.create_model>,
 'epochs': 3,
 'first_layer_nodes': 500,
 'last_layer_nodes': 50,
 'n_layers': 2}


### Build model

In [ ]:
def build_model(hp):
    
    """
    Returns a complied keras model ready for keras-tuner gridsearch algorithms 
    """
    
    model = Sequential()
    
    # hidden layer
    model.add(Dense(units=hp.get('units'),activation=hp.get("activation")))
    
    # output layer
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=Adam(hp.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model
  

In [ ]:
hp = HyperParameters()
hp.Int('units', min_value=32, max_value=512, step=32)
hp.Choice('learning_rate',values=[1e-1, 1e-2, 1e-3])
hp.Choice('activation',values=["relu", "sigmoid"])

'relu'

##Gridsearch Optimization


In [ ]:
hyper_parameters = {
    "units": np.arange(32, 512, 32).tolist(), # tolist() or error
    "learning_rate": [1e-1, 1e-2, 1e-3],
    "activation":["relu", "sigmoid"]
}

In [ ]:
hyper_parameters

{'activation': ['relu', 'sigmoid'],
 'learning_rate': [0.1, 0.01, 0.001],
 'units': [32,
  64,
  96,
  128,
  160,
  192,
  224,
  256,
  288,
  320,
  352,
  384,
  416,
  448,
  480]}

In [ ]:
def build_model(units, learning_rate, activation):
    
    """
    Returns a complie keras model ready for keras-tuner gridsearch algorithms 
    """
    
    model = Sequential()
    
    # hidden layer
    model.add(Dense(units, activation=activation))
    
    # output layer
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model

## Apply Keras wrapper

In [ ]:
model = KerasClassifier(build_fn = build_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
start = time()

grid = GridSearchCV(estimator=model, 
                    param_grid=hyper_parameters, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)

grid_result = grid.fit(X_train, y_train)

end = time()

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 90 candidates, totalling 270 fits
1477/1563 [===========================>..] - ETA: 0s - loss: 0.7497 - accuracy: 0.7735

In [ ]:
total_run_time_in_miniutes = (end - start)/60
total_run_time_in_miniutes

In [ ]:
grid_result.best_params_

In [ ]:
best_model = grid_result.best_estimator_
test_acc = best_model.score(X_test, y_test)

In [ ]:
test_acc

## Using Keras hyperparam tool

In [ ]:
random_tuner = RandomSearch(
            build_model,
            objective='val_accuracy',
            max_trials=n_param_combos_to_sample, 
            seed=1234,
            hyperparameters=hp, # pass in hyperparameter dictionary
            directory='./keras-tuner-trial',
            project_name='random_search')

In [ ]:
random_tuner.search(X_train, y_train,
                    epochs=3,
                    validation_data=(X_test, y_test))

In [ ]:
# identify the best score and hyperparamter 
random_tuner.results_summary()

In [ ]:
max_trials=24
num_initial_points=5
beta=5.0

#### Using BayesianOptimization class

In [ ]:
bayesian_tuner = BayesianOptimization(
                    build_model,
                    objective='val_accuracy',
                    max_trials=max_trials,
                    hyperparameters=hp, # pass in our hyperparameter dictionary
                    num_initial_points=num_initial_points, 
                    beta=beta, 
                    seed=1234,
                    directory='./keras-tuner-trial',
                    project_name='bayesian_optimization_4')

In [ ]:
bayesian_tuner.search(X_train, y_train,
               epochs=3,
               validation_data=(X_test, y_test))

In [ ]:
bayesian_tuner.results_summary()